# Project 1: Data Exploration, Visualization and Dimensionality Reduction
## Animal Shelter Outcomes: A Classification Problem
#### Alex Matsunami, RJ Smith, Cory Nichols

## Business Understanding

## Data Meaning and Data Types  (Add Info Here)

## Data Preprocessing: Transformation and Statistics 

In [24]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler,Imputer
import seaborn as sns
%matplotlib inline

In [25]:
df = pd.read_csv('~/desktop/train.csv')

In [26]:
df.head(3)

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White


In [27]:
print 'Data types and counts:\n\n', df.info(), '\n\n'
print 'Data Descriptions: Counts, Frequency'

df.describe()

Data types and counts:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26729 entries, 0 to 26728
Data columns (total 10 columns):
AnimalID          26729 non-null object
Name              19038 non-null object
DateTime          26729 non-null object
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
dtypes: object(10)
memory usage: 2.2+ MB
None 


Data Descriptions: Counts, Frequency


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,26729,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,26729,6374,22918,5,16,2,5,44,1380,366
top,A705677,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,1,136,19,10769,7816,15595,9779,3969,8810,2824


#### Data Cleanup

In [28]:
print 'Number of nulls in the data set: \n', df.isnull().sum()

Number of nulls in the data set: 
AnimalID              0
Name               7691
DateTime              0
OutcomeType           0
OutcomeSubtype    13612
AnimalType            0
SexuponOutcome        1
AgeuponOutcome       18
Breed                 0
Color                 0
dtype: int64


In [29]:
del df['OutcomeSubtype'], df['AnimalID'] # drop extraneous data  columns
df = df.dropna(subset=['SexuponOutcome']) # drop the one nan row in the sex column. Not a large impact to analysis.
df = df.reset_index()

print 'Number of unique, non nan ages:', len(np.unique(df.AgeuponOutcome[~pd.isnull(df['AgeuponOutcome'])])) 


Number of unique, non nan ages: 44


In [30]:
from collections import Counter
from random import randint

randChooser = []
for i in ['AgeuponOutcome', 'SexuponOutcome']:
    randChooser.append(Counter(df[i].values).most_common(3)) # get the 3 most common categories

df['AgeuponOutcome'] = [randChooser[0][randint(0,2)][0] # randomly pick one of the three categories and replace null
                        if pd.isnull(i) or i.lower() == '0 years' else i for i in df['AgeuponOutcome'].values]
 
df['SexuponOutcome'] = [randChooser[1][randint(0,2)][0]\
                        if i.lower() == 'unknown' else i for i in df['SexuponOutcome'].values]

In [31]:
df.head()

,index,Name,DateTime,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,0,Hambone,2014-02-12 18:22:00,Return_to_owner,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,1,Emily,2013-10-13 12:44:00,Euthanasia,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,2,Pearce,2015-01-31 12:28:00,Adoption,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,3,NaN,2014-07-11 19:09:00,Transfer,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,4,NaN,2013-11-15 12:52:00,Transfer,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


## Data Creation and Transformation

In [32]:
print 'There are %d unique names in the data set' % len(np.unique(df['Name'].values))

# 6375 unique names. That is quite a few names. Let's generalize into a binary feature called has_name instead:
df['has_name'] = [0 if pd.isnull(i) else 1 for i in df['Name']]

# Further, let's split out the intactness of an animal: whether or not the animal has been spayed or neutered
# as well as the gender:
df['intact'] = [i.split()[0] for i in df['SexuponOutcome']]
df['gender'] = [i.split()[1] for i in df['SexuponOutcome']]

# we will also convert ageuponoutcome into days:
ageMap = {'year': 365, 
          'week': 7, 
          'month': 30,
          'day': 1} 

df['AgeuponOutcome'] = [i.rstrip('s').split() for i in df['AgeuponOutcome'].values] # clean text to keep mapping simple

ageList = []
for i in df['AgeuponOutcome'].values:
    for k,v in ageMap.items():
        if i[1] == k:
            ageList.append(int(i[0]) * v)
df['AgeuponOutcome'] = ageList
df.info()

There are 6375 unique names in the data set
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26728 entries, 0 to 26727
Data columns (total 12 columns):
index             26728 non-null int64
Name              19037 non-null object
DateTime          26728 non-null object
OutcomeType       26728 non-null object
AnimalType        26728 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26728 non-null int64
Breed             26728 non-null object
Color             26728 non-null object
has_name          26728 non-null int64
intact            26728 non-null object
gender            26728 non-null object
dtypes: int64(3), object(9)
memory usage: 2.7+ MB


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26728 entries, 0 to 26727
Data columns (total 12 columns):
index             26728 non-null int64
Name              19037 non-null object
DateTime          26728 non-null object
OutcomeType       26728 non-null object
AnimalType        26728 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26728 non-null int64
Breed             26728 non-null object
Color             26728 non-null object
has_name          26728 non-null int64
intact            26728 non-null object
gender            26728 non-null object
dtypes: int64(3), object(9)
memory usage: 2.7+ MB


In [34]:
# lets also identify potentially 'aggressive' breeds using a bit of regex and top 10 aggressive breeds list: 
# http://www.therichest.com/rich-list/the-biggest/the-worlds-10-most-dangerous-dog-breeds/?view=all

import re

agg_breeds = ['Bull','Doberman','Rottweiler','Husky','German','Boxer','Malamute','Dane']

df['Breed'] = [re.split('\W+',i) for i in df['Breed']]

aggColumn = []
for i in df['Breed'].values:
    if len(agg_breeds) != len(set(agg_breeds).difference(i)):
        aggColumn.append((1))
    else:
        aggColumn.append(0)       
df['is_aggressive'] = aggColumn


In [35]:
# let's also grab only the primary color using same regex approach and reduce the number of categories
colorMap = {'exotic':['Agouti','Calico','Lynx','Tricolor','Tortie','Flame','Torbie'],
            'dark':['Black','Chocolate','Liver','Sable','Blue','Seal','Red'],
            'neutral':['Gray','Silver','Tan','Buff','Brown','Fawn'],
            'light':['Cream','Pink','White','Yellow','Lilac','Ruddy','Apricot','Orange','Gold']}

df['primary_color'] = [re.split('\W+',i)[0] for i in df['Color']]

for idx, color in enumerate(df['primary_color'].values):
    for k,v in colorMap.items():
        if color in v:
            df.set_value(idx,'primary_color', k)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26728 entries, 0 to 26727
Data columns (total 14 columns):
index             26728 non-null int64
Name              19037 non-null object
DateTime          26728 non-null object
OutcomeType       26728 non-null object
AnimalType        26728 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26728 non-null int64
Breed             26728 non-null object
Color             26728 non-null object
has_name          26728 non-null int64
intact            26728 non-null object
gender            26728 non-null object
is_aggressive     26728 non-null int64
primary_color     26728 non-null object
dtypes: int64(4), object(10)
memory usage: 3.1+ MB
